<a href="https://colab.research.google.com/github/Hou-jing/-repository-experiment/blob/main/%E6%96%87%E6%9C%AC%E4%BA%8C%E5%88%86%E7%B1%BB%E9%97%AE%E9%A2%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/My Drive/test.txt'

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
def file_process(file_path):
    with open(file_path,'r',encoding='utf_8') as f:
        labels=[]
        texts=[]
        for line in f:
            line=line.split(' ')
            labels.append(line[0])
            texts.append(line[1])     
    return labels,texts
def describe_file(file_path):
    labels,texts=file_process(file_path)
    file_df=pd.DataFrame({'label':labels,'text':texts})
    file_length=file_df['text'].apply(lambda x:len(x))
    return file_df
test_df=describe_file(path)
print(test_df)

In [ ]:
!pip install transformers
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")


In [16]:
test_text=[text for text in test_df['text']]
print(type(test_text))
encoded_input = tokenizer(test_text,padding=True ,return_tensors='tf')#tokenizer 处理（list(str）或者str type)
output = model(encoded_input)
print(output.keys())


<class 'list'>
odict_keys(['last_hidden_state', 'pooler_output'])


In [40]:
import tensorflow as tf
y_test=[int(label) for label in test_df['label']]
y_train=output['pooler_output']
y_test=tf.convert_to_tensor(y_test, dtype=tf.float32)
# y_test=torch.FloatTensor(y_test)
print(output['pooler_output'].shape,len(y_test))
print(type(y_test),type(y_train))

(50, 768) 50
<class 'tensorflow.python.framework.ops.EagerTensor'> <class 'tensorflow.python.framework.ops.EagerTensor'>


In [41]:
from keras.models import Sequential,Model
from keras.layers import LSTM, Dense, Embedding, Dropout,Input
from tensorflow.keras.optimizers import Adam
def build_classifier_model():
  x_input=Input(shape=(768,))
  x_out=Dense(4,activation='relu')(x_input)
  x_out=Dense(1,activation='softmax')(x_out)
  return Model(x_input,x_out)
classifier_model=build_classifier_model()
print(classifier_model.summary())
classifier_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
classifier_model.fit(y_train,y_test,epochs=2)


Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 768)]             0         
                                                                 
 dense_10 (Dense)            (None, 4)                 3076      
                                                                 
 dense_11 (Dense)            (None, 1)                 5         
                                                                 
Total params: 3,081
Trainable params: 3,081
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/2
2/2 [==============================] - 1s 8ms/step - loss: 0.0000e+00 - accuracy: 0.4800
Epoch 2/2
2/2 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - accuracy: 0.4800
